In [ ]:
!pip install azure-ai-formrecognizer --pre

In [64]:
import os
from azure.core.exceptions import ResourceNotFoundError
from azure.ai.formrecognizer import FormRecognizerClient
from azure.ai.formrecognizer import FormTrainingClient
from azure.core.credentials import AzureKeyCredential

In [65]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__


In [66]:
endpoint = "https://formrecogblogdemo.cognitiveservices.azure.com/"
key = "5adb3c0fdc90418a91dbd74f8d99446c"

In [67]:
form_recognizer_client = FormRecognizerClient(endpoint, AzureKeyCredential(key))
form_training_client = FormTrainingClient(endpoint, AzureKeyCredential(key))

In [99]:
def print_result(invoices, blob_name):
    for idx, invoice in enumerate(invoices):
        print("--------Recognizing invoice {}--------".format(blob_name))
        vendor_name = invoice.fields.get("VendorName")
        if vendor_name:
            print("Vendor Name: {} has confidence: {}".format(vendor_name.value, vendor_name.confidence))
        vendor_address = invoice.fields.get("VendorAddress")
        if vendor_address:
            print("Vendor Address: {} has confidence: {}".format(vendor_address.value, vendor_address.confidence))
        customer_name = invoice.fields.get("CustomerName")
        if customer_name:
            print("Customer Name: {} has confidence: {}".format(customer_name.value, customer_name.confidence))
        customer_address = invoice.fields.get("CustomerAddress")
        if customer_address:
            print("Customer Address: {} has confidence: {}".format(customer_address.value, customer_address.confidence))
        customer_address_recipient = invoice.fields.get("CustomerAddressRecipient")
        if customer_address_recipient:
            print("Customer Address Recipient: {} has confidence: {}".format(customer_address_recipient.value, customer_address_recipient.confidence))
        invoice_id = invoice.fields.get("InvoiceId")
        if invoice_id:
            print("Invoice Id: {} has confidence: {}".format(invoice_id.value, invoice_id.confidence))
        invoice_date = invoice.fields.get("InvoiceDate")
        if invoice_date:
            print("Invoice Date: {} has confidence: {}".format(invoice_date.value, invoice_date.confidence))
        invoice_total = invoice.fields.get("InvoiceTotal")
        if invoice_total:
            print("Invoice Total: {} has confidence: {}".format(invoice_total.value, invoice_total.confidence))
        due_date = invoice.fields.get("DueDate")
        if due_date:
            print("Due Date: {} has confidence: {}".format(due_date.value, due_date.confidence))


In [94]:
# Create the BlobServiceClient object which will be used to create a container client
connect_str = "DefaultEndpointsProtocol=https;AccountName=storageaccountaiaprab2d;AccountKey=NOYciUciFe/tJzEuSoVDPJt1MOO/gpCnZtZbyzxvyfGVK8+I5XDU8IvFZi/RhnusAi73MqC97/R93d6/e+dkJg==;EndpointSuffix=core.windows.net"

blob_service_client = BlobServiceClient.from_connection_string(connect_str)

container_client = blob_service_client.get_container_client("train")
invoiceUrlBase = container_client.primary_endpoint
print(invoiceUrlBase)

https://storageaccountaiaprab2d.blob.core.windows.net/train


In [98]:
print("\nProcessing blobs...")

# List the blobs in the container
blob_list = container_client.list_blobs()
for blob in blob_list:
    invoiceUrl = f'{invoiceUrlBase}/{blob.name}'
    print(invoiceUrl)
    poller = form_recognizer_client.begin_recognize_invoices_from_url(invoiceUrl)
    invoices = poller.result()
    print_result(invoices, blob.name)


Processing blobs...
https://storageaccountaiaprab2d.blob.core.windows.net/train/Invoice103563.pdf


TypeError: 'str' object cannot be interpreted as an integer